# 初始化

In [4]:
from datetime import date
import json

import pandas

from shapely.geometry import box
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

username = ""
password = ""

with open('profile.json', 'r', encoding='utf-8') as file:
    profile = json.load(file)
    username = profile["username"]
    password = profile["password"]

# connect to the API
# https://identity.dataspace.copernicus.eu/auth
api = SentinelAPI(username, password, 'https://apihub.copernicus.eu/apihub')
download_dir = '../data/S2A'

# 查询数据

In [6]:
# https://sentinelsat.readthedocs.io/en/stable/
bbox = list()
date_range = list()

with open('query_settings.json', 'r', encoding='utf-8') as file:
    query_settings = json.load(file)
    bbox = query_settings["bbox"]
    date_range = query_settings["date"]

date_range = [date.replace('-', '') for date in date_range]
polygon = box(minx=bbox[0], maxx=bbox[2], miny=bbox[1], maxy=bbox[3])
wkt = polygon.wkt

# search by polygon, time, and Hub query keywords
# footprint = geojson_to_wkt(read_geojson('map.geojson'))
footprint = wkt
products = api.query(footprint,
                     date = (date_range[0], date_range[1]),
                     platformname = 'Sentinel-2',
                     order_by = 'cloudcoverpercentage',
                     processinglevel = 'Level-2A',
                     cloudcoverpercentage = (1, 20))

# convert to Pandas DataFrame
products_df = api.to_dataframe(products)

products_df_sorted = products_df.sort_values(['cloudcoverpercentage', 'ingestiondate'], ascending=[True, True])
products_df_sorted = products_df_sorted.head(5)



products = products_df_sorted

ConnectionError: HTTPSConnectionPool(host='apihub.copernicus.eu', port=443): Max retries exceeded with url: /apihub/search?format=json&rows=100&start=0&orderby=cloudcoverpercentage%20asc&q=beginPosition%3A%5B%222021-03-25T00%3A00%3A00Z%22+TO+%222021-04-15T00%3A00%3A00Z%22%5D+cloudcoverpercentage%3A%5B%221%22+TO+%2220%22%5D+platformname%3A%22Sentinel-2%22+processinglevel%3A%22Level-2A%22+footprint%3A%22Intersects%28POLYGON+%28%28108.65+21.8%2C+108.65+21.9%2C+108.45+21.9%2C+108.45+21.8%2C+108.65+21.8%29%29%29%22 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001476B2631C0>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))

# 数据下载

In [ ]:
api.download_all_quicklooks(products, directory_path="../data/S2A/quicklook")

# # download all results from the search
# api.download_all(products)


Beginning download of zipped output...
Data request 5000005879298 of  1  order(s) is downloaded.
Download complete
Total number of data order requests is  1  for  1  granules.
Data request  1  of  1  is submitting to NSIDC
order ID:  5000005879303
Initial status of your order request at NSIDC is:  processing
Your order status is still  processing  at NSIDC. Please continue waiting... this may take a few moments.
Your order is: complete
NSIDC returned these messages
['Granule 263960249 contained no data within the spatial and/or temporal '
 'subset constraints to be processed']
Beginning download of zipped output...
Unable to download  5000005879303 . Check granule order for messages.
Download complete
下载完成
